<a href="https://colab.research.google.com/github/DonErnesto/adventofcode/blob/main/advent_of_code_day13_mirrors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import numpy as np

Get data from the repository

In [2]:
data_url = r"https://raw.githubusercontent.com/DonErnesto/adventofcode/main/input_mirrors.txt"

In [3]:
import urllib.request
with urllib.request.urlopen(data_url) as f:
    # data is a list of strings, each string a row
    data = f.read().decode('utf-8').split('\n')

convert data to a list of 2-D np arrays. Each array is a field to solve

In [4]:
def convert_data(data):
  mapper = {'.':0, '#':1}
  list_of_rows = []
  list_of_arrays = [] # this will contain the final numpy arrays
  for row in data:
    if row == '':
      list_of_arrays.append(np.array(list_of_rows))
      list_of_rows = []
    else:
      list_of_rows.append([mapper.get(el) for el in row])
  return list_of_arrays

list_of_arrays = convert_data(data)

mirror finding algorithm



In [5]:
def generate_comparison_indices(index, length):
  lower_index_list = np.arange(index-1, -1, -1)
  upper_index_list = np.arange(index, length)
  return list(zip(lower_index_list, upper_index_list))
assert generate_comparison_indices(4, 5) == [(3, 4)] # boundaries are tricky.

In [6]:
def search_mirror_plane(field, axis=0):
  for potential_mirror_plane in range(1, field.shape[axis]):
    is_mirror_plane = True
    # note that the location of a mirror plane is defined by the number of rows above it
    while is_mirror_plane:
      for upper_index, lower_index in generate_comparison_indices(potential_mirror_plane, field.shape[axis]):
        if axis == 0:
          test_equality = all(field[upper_index, :] == field[lower_index, :])
        else:
          test_equality = all(field[:, upper_index] == field[:, lower_index])
        if not test_equality:
          is_mirror_plane = False
      if is_mirror_plane:
        mirror_plane = potential_mirror_plane
        return mirror_plane
  return 0 # meaning: no mirror plane found


In [7]:
# define some tests
def test_row_middle():
  test_field = np.array([[0, 1, 0, 2], [1, 0, 0, 1], [1, 0, 0, 1], [0, 1, 0, 2]])
  assert search_mirror_plane(test_field, axis=0) == 2
  assert search_mirror_plane(test_field, axis=1) == 0

def test_col_end():
  # "edge" case
  test_field = np.array([[0, 1, 0, 1, 1]])
  assert search_mirror_plane(test_field, axis=1) == 4

def test_col():
  test_field = np.array([[0, 1, 1, 0], [1, 1, 1, 1], [1, 0, 0, 1]])
  assert search_mirror_plane(test_field, axis=0) == 0
  assert search_mirror_plane(test_field, axis=1) == 2


test_row_middle()
test_col_end()
test_col()

In [8]:
# Function that runs on all fields and gives the solution

def solve_puzzle(list_of_arrays):
  total_result = 0
  for field in list_of_arrays:
    row_result = 100 * search_mirror_plane(field, axis=0)
    column_result = search_mirror_plane(field, axis=1)
    total_result += row_result + column_result
  return total_result

Final test: on the website example

In [9]:
test_data = """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
"""
test_list_of_arrays = convert_data(test_data.split('\n'))

assert solve_puzzle(test_list_of_arrays) == 405  # example from website

In [10]:
# generate the final result
solve_puzzle(list_of_arrays)

30535

That's the right solution!